In [16]:
import os
import pickle
import numpy as np
import tensorflow as tf
import madmom
import mir_eval
from sklearn.model_selection import KFold

from modules.labels import get_label_vector
from modules.madmom_cnn_prep import cnn_preprocessor
from datasets import Dataset
from modules.analysis_funcs import get_idx_to_fold, get_segmented_data, get_test_peaks
from analyze_detection import evaluate
from modules.energy_based import legato_mg, onsets_threshold_gate

%load_ext autoreload
%autoreload 2

FPS = 100
CONTEXT = 7

# Load Madmom normalization
def cnn_normalize(frames):
    inv_std = np.load("models/bock2013pret_inv_std.npy")
    mean = np.load("models/bock2013pret_mean.npy")
    frames_normalized = (frames - np.reshape(mean, (1,80,3)))*np.reshape(inv_std, (1,80,3))
    return frames_normalized

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of datasets failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/simonfal/kth-master/exjobb/beyondPianoRoll/datasets/__init__.py", line 63
    class Dataset:
    ^


In [17]:
frame = madmom.audio.signal.FramedSignalProcessor(frame_size=2048, hop_size=441)
stft = madmom.audio.stft.STFTProcessor()
spect = madmom.audio.spectrogram.SpectrogramProcessor()
proc = madmom.processors.SequentialProcessor([frame, stft, spect])

In [33]:
ds0 = Dataset("initslurtest")
ds1 = Dataset("slurtest_add_1")
ds2 = Dataset("slurtest_add_2")

audio_fnames = ds2.get_audio_paths() #+ ds1.get_audio_paths()
#label_fnames = ds1.get_annotation_paths() #+ ds1.get_annotation_paths()

audios = [madmom.audio.signal.load_wave_file(filename)[0] for filename in audio_fnames]
sample_rates = [madmom.audio.signal.load_wave_file(filename)[1] for filename in audio_fnames]
#onset_schedules = [np.loadtxt(label_fname, usecols=0) for label_fname in label_fnames]

/usr/local/lib/python3.9/site-packages/madmom/audio/signal.py:457: UserWarning: Deprecated as of version 0.16. Please use madmom.io.audio.load_wave_file instead. Will be removed in version 0.18.
  warnings.warn('Deprecated as of version 0.16. Please use madmom.io.audio.'
/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


In [35]:
audio_lengths_sec = [len(audio)/sr for audio,sr in zip(audios, sample_rates)]
#iois = [np.ediff1d(onsets) for onsets in onset_schedules]
#ioi_spreads = [np.std(np.ediff1d(onsets)) for onsets in onset_schedules]
#onset_number = [len(onsets) for onsets in onset_schedules]
#mean_ioi = np.sum([np.sum(ioi) for ioi in iois])/np.sum(onset_number)
print(len(audios))
print(np.sum(audio_lengths_sec)/60, "s")
print(.819142526455026*60)
print(np.sum(onset_number))
print(np.mean(audio_lengths_sec))
print(mean_ioi)

13
8.819142526455026 s
49.14855158730156
1380
40.70373473748474
0.4011483079710145


In [11]:
random_seed = 119
n_splits =  5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=True)
kf_gen = list(kf.split(np.arange(len(audio_fnames))))

In [12]:
base_path = "results/cnn-training-220409/"
folds_path = base_path + "folds.pkl"

model_name = "added-sample-gen-nostandard"

folds = kf_gen
#with open(folds_path, "rb") as f:
    #folds = pickle.load(f)

itf = get_idx_to_fold(folds)

neural = False
TOL = 0.025


In [13]:
neural=False
CD_list = []
FN_list = []
FP_list = []
for r in range(len(itf.keys())):
    fold = itf[r]
    rec_name = os.path.basename(audio_fnames[r])
    x = get_segmented_data(audio_fnames[r])
    if neural:
        model = tf.keras.models.load_model(base_path + "fold_" + str(fold) + "_" + model_name + "_model")
        out = model.predict(x)
        peaks = get_test_peaks(out, 1./FPS)
    else:
        leg_on, leg_val = legato_mg(audio_fnames[r], rel_delta=0.2)
        peaks = onsets_threshold_gate(leg_on, leg_val, 1.0)
    
    [CD,FN,FP,doubles,merged] = evaluate(onset_schedules[r], peaks, tol_sec=TOL)
    CD_list.append(CD)
    FN_list.append(FN)
    FP_list.append(FP)

    scores = mir_eval.onset.evaluate(onset_schedules[r], peaks, window=TOL)
    print(rec_name + "\t" + "F-score: {:.2f}".format(100*scores["F-measure"]))



IndexError: index 1698 is out of bounds for axis 0 with size 1697

In [40]:
np.sum(CD_list)/(np.sum(CD_list)+.5*(np.sum(FP_list) + np.sum(FN_list)))

0.8505654281098546